In [1]:
%run utilities.ipynb

<IPython.core.display.Javascript object>

## Numerical Method
---
Many physical systems can be expressed as inital value problems (IVP). These usually have the form of a set of coupled ordinary differential equations (ODEs), and given initail conditions they can be solved numerically. Standard Python scientific library, such as `scipy.optimize` with functions such as `oedint()`, `ode()` or `solve_ivp()` can be used for this purpose. Here, we will define our own class of ODE solver. This will enable different methods, such as explicit and implict Runge Kutta methods to be implemented to tackle a wider range of IVP.

Object oriented programming is particularly suited for this type of applications. We will define a master class titled `ODESolver`, which will contain instances to set the initial conditions of the IVP, advance the solution in time and finally solve the complete initial value problem.

In [2]:
class ODESolver(object):
    def __init__(self, f):
        # check that f is a function
        if not callable(f):
            raise TypeError('f is a %s, not a function' % type(f))
        self.f = lambda t, q: np.asarray(f(t, q), float)

    def advance(self):
        # Advance solution one time step, this depends on the method chosen
        raise NotImplementedError

    def set_initial_condition(self, q0):
        # single ODE
        if isinstance(q0, (float, int)):
            self.neq = 1
            q0 = float(q0)
        else:
            q0 = np.asarray(q0)
            self.neq = q0.size
        self.q0 = q0
        # Check that f returns correct length:
        try:
            f0 = self.f(0, self.q0)
        except IndexError:
            raise IndexError('Index of q out of bounds in f(t, q) func. Legal indices are %s' %\
                             (str(range(self.neq))))
        if f0.size != self.neq:
            raise ValueError('f(t, q) returns %d components, while q has %d components' %\
                             (f0.size, self.neq))

    def solve(self, t):
        # chech that the t array is really an array
        if isinstance(t, (float, int)):
            raise TypeError('solve: t is not a sequence')
        # converts the t array to a numpy array
        self.t = np.asarray(t)
        # chach that the array has at least two time points
        if self.t.size <= 1:
            raise ValueError('ODESolver.solve requires a time array with at least 2 time points')

        # get number of steps from size of the time array
        Nsteps = self.t.size

        # initialise solution vector for a single ODE
        if self.neq == 1:
            self.q = np.zeros(Nsteps)
        # initialise solution vector for a system of ODEs
        else:
            self.q = np.zeros((Nsteps, self.neq))

        # populate with initial condition
        self.q[0] = self.q0

        # advance the solution in time
        for n in range(Nsteps-1):
            self.n = n
            # this will depend on the algorithm chosen
            self.q[n+1] = self.advance()
        return self.t, self.q

### Standard Runge Kutta ($3^{rd}$, explicit)
---
Given initial data $\textbf{q}_{n}$ for an ODE problem at location $t_n$, using an evenly spaced grid with spacing $\Delta t$ (so that $t_{n+1} = t_{n} +\Delta t$), the algorithm can be written

$$
\textbf{k}^{(1)} = \textbf{f}(t_{n}, \textbf{q}_{n}),
$$

$$
\textbf{k}^{(2)} = \textbf{f}\left(t_{n}+ \frac{\Delta t}{2}, \textbf{q}_{n} + \frac{\Delta t}{2}\textbf{k}^{(1)}\right),
$$

$$
\textbf{k}^{(3)} = \textbf{f}(t_{n} + \Delta t, \textbf{q}_{n} + 
\left[-\textbf{k}^{(1)} + 2\textbf{k}^{(2)}\right]),
$$

$$
\textbf{q}_{n+1} = \textbf{q}_{n} +\frac{\Delta t}{6}\left(
\textbf{k}^{(1)} + 4\textbf{k}^{(2)} + \textbf{k}^{(3)}\right)
$$


In [3]:
class RungeKutta3(ODESolver):
    def advance(self):
        """
        Advance the solution one time step using the 3rd order Runge Kutta.

        Butcher Tableau:

             0  |  0   0    0
            1/2 | 1/2  0    0
             1  | -1   2    0
            -----------------
                | 1/6 2/3 1/6

        """

        f, t, q, n = self.f, self.t, self.q, self.n
        dt = t[n+1] - t[n]
        dt2 = dt / 2.0

        # intermediate steps
        K1 = f(t[n], q[n])
        K2 = f(t[n] + dt2, q[n] + dt2 * K1)
        K3 = f(t[n] + dt, q[n] + dt * (-K1 + 2 * K2))

        # advance the solution one time step
        q_new = q[n] + (dt / 6.0) * (K1 + 4 * K2 + K3)

        return q_new

### Gauss-Radan Runge Kutta ($3^{rd}$ implicit)
---
Given initial data $\textbf{q}_n$ for an ODE problem at location $t_n$, using an evenly spaced grid with spacing $\Delta t$ so that $t_{n+1} = t_{n} +\Delta t$), the algorithm can be written

$$
\textbf{k}^{(1)} = \textbf{f}\left(t_{n} + \frac{1}{3}\Delta t, 
\textbf{q}_{n} + \frac{\Delta t}{12}\left[5\textbf{k}^{(1)} - \textbf{k}^{(2)}\right]\right)
$$

$$
\textbf{k}^{(2)} = \textbf{f}\left(t_{n} + \Delta t, 
\textbf{q}_{n} + \frac{\Delta t}{4}\left[3\textbf{k}^{(1)} + \textbf{k}^{(2)}\right]\right)
$$

$$
\begin{equation}
\textbf{q}_{n+1} = \textbf{q}_{n} + \frac{\Delta t}{4}\left(3\textbf{k}^{(1)} + \textbf{k}^{(2)}\right)
\end{equation}
$$

We note that the definition of $\textbf{k}^{(i)}$ is implicit, so finding the update terms will involve solving a nonlinear algebraic equation. The implementation must solve the nonlinear algebraic system defined by the equations for $\textbf{k}^{(1)}$ and $\textbf{k}^{(2)}$ : with this solution the update defined in the equation for $\textbf{q}_{n+1}$ can be computed explicitly. To solve the implicit problem above we define

$$
\textbf{K}=\begin{pmatrix}
        \textbf{k}^{(1)} \\
        \textbf{k}^{(2)}
    \end{pmatrix}.
$$

We then write the problem to be solved as $\textbf{K}=0$, where

$$
\textbf{F} = \textbf{K} - \begin{pmatrix}
        \textbf{f}(t_{n} + \frac{1}{3}\Delta t, 
\textbf{q}_{n} + \frac{\Delta t}{12}[5\textbf{k}^{(1)} - \textbf{k}^{(2)}])\\
        \textbf{f}(t_{n} + \Delta t, 
\textbf{q}_{n} + \frac{\Delta t}{4}[3\textbf{k}^{(1)} + \textbf{k}^{(2)}])
    \end{pmatrix}.
$$

Given some initial guess for $\textbf{K}$, which corresponds to initial guesses for $\textbf{k}^{(1, 2)}$, this can be solved via the fsolve or root function from the `scipy` library. A simple initial guess for $\textbf{K}$ sets $\textbf{K}^{(1)} = \textbf{f}(t_{n} + \Delta t/3, \textbf{q}_{n})$ and $\textbf{K}^{(1)} = \textbf{f}(t_{n} + \Delta t, \textbf{q}_{n})$.

In [4]:
class GaussRadanRungeKutta(ODESolver):
    def advance(self):
        """
        Advance the solution one time step using the 3rd order Implicit Gauss
        Radan Runge Kutta.

        Butcher Tableau:

            1/3 | 5/12 -1/12
             1  | 3/4   1/4
            ----------------
                | 3/4   1/4

        """
        f, t, q, n, neq = self.f, self.t, self.q, self.n, self.neq
        dt = t[n+1] - t[n]
        dt3 = dt / 3.0

        # implicit step
        def K_func(attributes):

            # attributes contains all the Ks required
            K = attributes

            F1 = K[:neq] - f(t[n] + dt3, q[n] + dt / 12.0 * (5 * K[:neq] - K[neq:]))
            F2 = K[neq:] - f(t[n] + dt, q[n] + dt / 4.0 * (3 * K[:neq] + K[neq:]))

            return np.array([F1, F2]).ravel()

        # initial guess for fsolve
        K_init = np.array([f(t[n] + dt3, q[n]),
                           f(t[n] + dt,  q[n])])

        # solve the implicit step
        K = fsolve(K_func, K_init)

        # advance the solution one time step
        q_new = q[n] + (dt / 4.0) * (3 * K[:neq] + K[neq:])

        return q_new